#  Load Playlists.cvs


In [ ]:
import pandas as pd
import os

import glob
file_paths = glob.glob(".\data\*.csv")

dfs = [] # an empty list to store the data frames
for file_path in file_paths:
    data = pd.read_csv (file_path)
    data['playlist_name'] = os.path.basename(file_path).split('.')[-2]
    dfs.append(data) # append the data frame to the list

videos = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

videos

In [ ]:
videos = videos.set_axis(["video_id", "video_date", 'playlist_name'], axis=1)
videos['video_id'] = videos['video_id'].apply(lambda x: str(x).strip())
videos

#  GOOGLE API - request


In [ ]:
from googleapiclient.discovery import build
from SECRETS import YOUTUBE_API_KEY
import json
API_KEY = YOUTUBE_API_KEY

In [ ]:
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_video_info(id):
    request = youtube.videos().list(
        part='snippet,contentDetails,statistics,topicDetails,localizations', 
        id=id
        )
    return request.execute()

In [ ]:
videos['res'] = videos['video_id'].apply(get_video_info)

# Extracting data from playlist.csv and GOOGLE API response


## Playlist added date - from playlist.csv

In [ ]:
def string_to_date(string):
    return pd.to_datetime(string[:-4], dayfirst=True)
def get_month(date):
    return int(date.month)
def get_year(date):
    return int(date.year)
def get_weekday(date):
    return date.day_name()

In [ ]:
videos['video_date'] = videos['video_date'].apply(string_to_date)
videos['video_date_m'] = videos['video_date'].apply(get_month)
videos['video_date_y'] = videos['video_date'].apply(get_year)
videos['video_date_w'] = videos['video_date'].apply(get_weekday)
# videos

## Video info - from Google Api response:


### Basic text info

In [ ]:
def get_video_title(info):
    for item in info['items']:
        return item['snippet']['title']
              
def get_video_description(info):
    for item in info['items']:
        return item['snippet']['description']

def get_video_channel(info):
    for item in info['items']:
        return item['snippet']['channelTitle']

def get_video_topicCategories(info):
    for item in info['items']:
        if (topicDetails := item.get('topicDetails')) != None:
            return topicDetails.get('topicCategories')

def get_video_AudioLanguage(info):
    for item in info['items']:
        return item['snippet'].get('defaultAudioLanguage')

In [ ]:
videos['video_title'] = videos['res'].apply(get_video_title)
videos['video_description'] = videos['res'].apply(get_video_description)
videos['video_channel'] = videos['res'].apply(get_video_channel)
videos['video_topic'] = videos['res'].apply(get_video_topicCategories)
videos['video_lang'] = videos['res'].apply(get_video_AudioLanguage)

### Video lenght

In [ ]:
import isodate
def get_video_len(info):
    for item in info['items']:
        return isodate.parse_duration(item['contentDetails']['duration'])

def deltatime_to_seconds(len):
    return len.total_seconds()

In [ ]:
videos['video_len'] = videos['res'].apply(get_video_len)
videos['video_len_s'] = videos['video_len'].apply(deltatime_to_seconds)

### Video stats


In [ ]:
def get_viewCount(info):
    for item in info['items']:
        if (statistics := item.get('statistics')) != None:
            return statistics.get('viewCount')
def get_likeCount(info):
    for item in info['items']:
        if (statistics := item.get('statistics')) != None:
            return statistics.get('likeCount')
def get_commentCount(info):
    for item in info['items']:
        if (statistics := item.get('statistics')) != None:
            return statistics.get('commentCount')
def to_int(string):
    if string != None:
        return int(string)

In [ ]:
videos['video_viewCount'] = videos['res'].apply(get_viewCount)
videos['video_likeCount'] = videos['res'].apply(get_likeCount)
videos['video_commentCount'] = videos['res'].apply(get_commentCount)

videos['video_viewCount'] = videos['video_viewCount'].apply(to_int)
videos['video_likeCount'] = videos['video_likeCount'].apply(to_int)
videos['video_commentCount'] = videos['video_commentCount'].apply(to_int)

videos
# videos[['tytul', 'viewCount', 'likeCount', 'commentCount']].sort_values('viewCount', ascending=False)

#   Spotify API 

## Prepare api

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from SECRETS import SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET, redirect_uri=SPOTIFY_REDIRECT_URI))

## Requests

In [ ]:
def get_spotify_data(vid):
    # print(vid)
    if vid['video_title'] == None:
        # print('')
        # print('')
        # print({})
        # print([])
        return None,None
    # q = str(vid['video_title'])
    for q in [vid['video_title'] + ' ' + vid['video_channel'], vid['video_title']]:
        search_results = sp.search(q, limit=10, offset=0, type='track', market=None)
        if len(search_results['tracks']['items']):
            track_id = search_results['tracks']['items'][0]["id"]

            audio_features_results = sp.audio_features(tracks=[track_id])[0]
            if audio_features_results != None:
                track_features = dict((key, audio_features_results[key]) for key in ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])
                track_features['popularity'] = search_results['tracks']['items'][0]["popularity"]
            else:
                track_features = {}
            artist_id = search_results['tracks']['items'][0]["artists"][0]["id"]
            track_author = sp.artist(artist_id)
            track_genre = track_author["genres"]

            # print(vid['video_title'] + ' ' + vid['video_channel'])
            # print(search_results['tracks']['items'][0]["name"] +' - ' + search_results['tracks']['items'][0]["artists"][0]["name"])
            # print(track_features)
            # print(track_genre)

            return track_features, track_genre
    # print(vid['video_title'] + ' ' + vid['video_channel'])
    # print('')
    # print({})
    # print([])
    return None,None

videos['video_features'], videos['video_genre'] = zip(*videos[['video_title', 'video_channel']].apply(get_spotify_data, axis=1))
# poki = videos[['video_title','video_channel']].apply(get_spotify_data, axis=1)
#  videos[['video_features', 'video_genre']]
videos

In [ ]:
videos[['playlist_name','video_title', 'video_channel', 'video_genre', 'video_features']].to_excel('OUTPUT.xlsx')

#  Play history
## Load history + clean data

In [ ]:
history = pd.read_json('data\history.json')
history = history.drop(['header','products','details','description', 'activityControls', 'subtitles'], axis=1, errors='ignore')
history = history.rename(columns = {'time':'date'})
history = history.rename(columns=lambda x: 'history_'+x)
history = history[history['history_titleUrl'].notna()]
history.columns

 ## Extract data

In [ ]:
def extract_id(url):
    return str(url.replace('https://www.youtube.com/watch?v=', ''))
def extract_clear_title(title):
    return title.replace('Obejrzano: ', '')
def string_to_date_history(string):
    return pd.to_datetime(string[:10])

history['history_video_id'] = history['history_titleUrl'].apply(extract_id)
# history = history.drop(['history_titleUrl'], axis=1)

history['history_title'] = history['history_title'].apply(extract_clear_title)

history['history_date'] = history['history_date'].apply(string_to_date_history)
history['history_date_m'] = history['history_date'].apply(get_month)
history['history_date_y'] = history['history_date'].apply(get_year)
history['history_date_w'] = history['history_date'].apply(get_weekday)
history

#  JOIN - watch_history + playlist 

## Łączenie:

In [ ]:
playlist_history = pd.merge(
    history,
    videos,
    how="right",
    left_on='history_video_id',
    right_on='video_id',
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_history", "_videos"),
    copy=True,
    indicator=False,
    validate=None,
)
playlist_history

#  Analysis and graph generate \<in progress\> 


In [ ]:
import matplotlib.pyplot as plt

## Music on playlist analysis
### Num. of songs on each playlist

In [ ]:
to_plot = videos
to_plot = to_plot.groupby(['playlist_name']).count().filter(items=['video_id']).sort_values('video_id')
# print(to_plot)
to_plot = to_plot.plot(kind='bar',figsize = (12, 6), style='o-.', ylabel='count_video_id', title='Num. of songs on each playlist')
to_plot = to_plot.bar_label(to_plot.containers[0], fmt="%d")

### Num. of songs in each year

In [ ]:
to_plot = videos
to_plot = to_plot.groupby(['video_date_y', 'playlist_name'])
to_plot = to_plot.count().filter(items=['video_id'])
to_plot = to_plot.unstack()
to_plot.plot(kind='line',figsize = (12, 6), style='o-.', ylabel='count_video_id', grid=True, title = 'Num. of song added in each year')

### Num. of song added in given date

In [ ]:
to_plot = videos
to_plot = to_plot.groupby(['video_date_y','video_date_m', 'playlist_name'])
to_plot = to_plot.count().filter(items=['video_id'])#.unstack()
to_plot = to_plot.unstack()
to_plot.plot(kind='line',figsize = (25, 12), style='o--', ylabel='count_video_id', grid=True, title = 'Num. of song added in given date')

### Num. of song added in each month

In [ ]:
to_plot = videos
to_plot = to_plot.groupby(['video_date_m', 'playlist_name'])
to_plot = to_plot.count().filter(items=['video_id'])#.unstack()
to_plot = to_plot.unstack()
to_plot
to_plot.plot(kind='line',figsize = (12, 6),  style='o-.', ylabel='count_video_id', grid=True, title = 'Num. of song added in each month')

### Num. of song added by week day

In [ ]:
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
to_plot = videos.groupby(['video_date_w', 'playlist_name'])
to_plot.count().reindex(cats, level='video_date_w')[['video_id']].unstack().plot(kind='line',figsize = (16, 9),style='o-.', ylabel='count_video_id', grid=True, title = 'Num. of song added by weekday')


### Most play video_channel

In [ ]:
to_plot = videos.groupby(['video_channel'])
to_plot.count()[['video_id']].sort_values('video_id', ascending=False) #.plot(kind='line',figsize = (16, 9),style='o-.', ylabel='count_video_id', grid=True, title = 'Num. of song added by weekday')


### Top of 'video_len_s', 'video_likeCount', 'video_viewCount', 'video_commentCount', 'ratios' and mean of playlist

In [ ]:
# sort by one of 'video_len_s', 'video_likeCount', 'video_viewCount', 'video_commentCount'
q = videos
q['l/v_ratio'] = q['video_likeCount'] / q['video_viewCount']
q['c/v_ratio'] = q['video_commentCount'] / q['video_viewCount']
q['c/l_ratio'] = q['video_commentCount'] / q['video_likeCount']
q = q[['video_id','playlist_name','video_title', 'video_channel', 'video_len_s', 'video_likeCount', 'video_viewCount', 'video_commentCount', 'l/v_ratio', 'c/l_ratio', 'c/v_ratio']].dropna(subset=['video_title'])
top_of = 'c/l_ratio'
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 50000): 
    print(q.dropna(subset=[top_of]).sort_values(top_of, ascending = False))

q.groupby('playlist_name').mean()

### Sort by given music feature

In [ ]:
def to_series(x):
    print(x)
    return pd.Series(x)
features = videos

SORT_BY = 'video_feature_acousticness'

for index in features.index:
    dic = features.loc[index,'video_features']
    if dic != None:  
        for k, v in dic.items():
            features.loc[index,'video_feature_' + k] = v
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 50000): 
    print(features[features['playlist_name'] == 'Ex1'][['video_title',SORT_BY]].dropna().sort_values(SORT_BY, ascending=False))



### Mean for given feature

In [ ]:
features.groupby(['playlist_name']).mean().filter(items=['video_feature_danceability',	'video_feature_energy',	'video_feature_speechiness'	,'video_feature_acousticness',	'video_feature_instrumentalness',	'video_feature_liveness','video_feature_valence','video_feature_tempo','video_feature_popularity'])

### Count genres for each playlist

In [ ]:
videos['video_genre']

def to_series(x):
    print(x)
    return pd.Series(x)
genres = videos
df_list = []

for index in genres.index:
    li = genres.loc[index,'video_genre']
    if li != None:  
        for val in li:
            df = list(genres.loc[index,['video_id', 'video_title', 'playlist_name']]) + [val]
            df_list.append(pd.DataFrame([df], columns=['video_id', 'video_title', 'playlist_name', 'video_genre']))

genres_ans = pd.concat(df_list)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 50000): 
    print(genres_ans.groupby(['playlist_name', 'video_genre']).count()['video_id'].unstack().unstack().unstack().fillna(0).sort_values('📼 Tapes Colection 📼', ascending=False))

## history + videos graphs

### play count by year

In [ ]:
to_plot = playlist_history.dropna(subset=['history_date_y'])
to_plot['history_date_y'] = to_plot['history_date_y'].astype('int64')
to_plot['history_date_y'] = to_plot['history_date_y'].astype('str')
# to_plot
to_plot = to_plot.groupby(['history_date_y', 'playlist_name'])
to_plot = to_plot.count()['video_id']
to_plot = to_plot.unstack()
to_plot = to_plot.plot(kind='line',figsize = (12, 6), style='o-.', ylabel='count_hisotry_play', grid=True)

### Most played music in given year:

In [ ]:
YEAR = 2022
ans = playlist_history.groupby(['history_date_y', 'video_title'])
ans = ans.count().sort_values(['history_date_y', 'video_id'], ascending=False)
ans = ans.unstack()['video_id'].unstack().unstack()
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(ans.sort_values(YEAR,ascending = False)[YEAR])


### Most played music of all time:

In [ ]:
gk = playlist_history.groupby(['video_title'])
wynik = gk.count().sort_values(['video_id'], ascending=False)['video_id']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(wynik)




In [ ]:
nieogladniete = playlist_history[playlist_history['history_video_id'].isna()]
nieogladniete = nieogladniete.drop_duplicates(subset = ['video_id'])
nieogladniete[['video_title', 'video_id',  'video_channel', 'history_video_id', 'history_titleUrl']]


### Top played genres in given year

In [ ]:
playlist_history['video_genre']

def to_series(x):
    print(x)
    return pd.Series(x)
genres = playlist_history
df_list = []

for index in genres.index:
    li = genres.loc[index,'video_genre']
    if li != None:  
        for val in li:
            df = list(genres.loc[index,['video_id', 'video_title', 'playlist_name', 'history_date_y']]) + [val]
            df_list.append(pd.DataFrame([df], columns=['video_id', 'video_title', 'playlist_name', 'video_genre', 'history_date_y']))

genres_ans = pd.concat(df_list)

In [ ]:
year = 2022.0
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 50000): 
    print(genres_ans.groupby(['playlist_name','history_date_y', 'video_genre']).count()['video_id'].unstack().fillna(0).reset_index().rename_axis(None, axis=1)[['history_date_y', year]].sort_values(year, ascending=False)) #.sort_values('2022.0', ascending=True))